# О проекте


Задание относится к проекту UNICEF — международного подразделения ООН, чья миссия состоит в повышении уровня благополучия детей по всему миру. 


## Цель проекта

Отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

## Средства достижения целей

Реализовать достижение целей возможно с помощью модели, которая предсказывала бы результаты госэкзамена по математике для каждого ученика школы.

## Задача проекта

Чтобы определиться с параметрами будущей модели, нужно провести разведывательный анализ данных и составьте отчёт по его результатам.

## Импорт библиотек

In [ ]:
import warnings; warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import os
import re
import math

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from itertools import combinations

from scipy.stats import ttest_ind
from scipy.stats import norm
from scipy.stats import t
from statsmodels.stats import weightstats 

%matplotlib inline

### Инструменты для предобработки

In [ ]:
def column_content_analysis(column_name_list, column_type='str', more_than = 10):
    '''
        Функция для определения типовых параметров данных, а также определние "замусоренности".
        На вход принимает список столбцов датафрейма
    '''
    # проходим циклом по списку колонок    
    for column_name in column_name_list:
        print('-'*60)
        print('|\t' , 'Отчет по колонке [', column_name, ']', '\t'*5)
        print('-'*60)
        if column_name not in stud.columns:
            print('|  Внимание!!! Указанная колонка не найдена в датасете. \n')
            break

        col_count = stud[column_name].count()
        col_type = stud[column_name].dtype
        print('|  Тип данных:', col_type) 
        print('|  Заполнено значений:',  col_count, 'из', row_count)
        print('|  Отсутсвующие значения:', row_count - col_count)
        print('|  Полнота данных: ', round(col_count / row_count * 100 ,2), '%', sep='')
        print('|  Количество уникальных значений:', stud[column_name].nunique())

        print('|  Значений, встретившихся в столбце более', more_than,'раз:', 
              (stud[column_name].value_counts() > more_than).sum())

        if col_type == 'object':
            # Количество пробелов в начале в конце строки
            print('|  Количество раз пробелы найдены:')
            find_list = [len(re.findall(r'^\s+\w?', str(x))) for x in stud[column_name]]
            print('|    в начале строки:', sum(find_list))
            
            find_list = [len(re.findall(r'\w?\s+$', str(x))) for x in stud[column_name]]
            print('|    в конце  строки:', sum(find_list))
            
            find_list = [len(re.findall(r'[^\s\d\w]', str(x))) for x in stud[column_name]]
            print('|  Количество найденных специальных символов:', sum(find_list))
        
        if col_type in ['int', 'float']:
            print('|  Медиана: ', stud[column_name].median())
            
            perc25 = stud[column_name].describe().loc['25%']
            perc75 = stud[column_name].describe().loc['75%']
            iqr = perc75 - perc25
            range_left = perc25 - 1.5 * iqr
            range_right = perc75 + 1.5 * iqr
            
            # Границы выбросов
            print('|  IQR: ', iqr, '; границы выбросов: [', round(range_left, 2), '; ', round(range_right,2),']' , sep='')
            out_of_range_left = stud[column_name].loc[stud[column_name] < range_left]
            out_of_range_right = stud[column_name].loc[stud[column_name] > range_right]
            out_of_range = len(out_of_range_left) + len(out_of_range_right)
            if out_of_range > 0:
                print('|  Данные СОДЕРЖАТ выбросы, количество значений: ', out_of_range)
                if len(out_of_range_left) > 0:
                    print('|    Слева  от IQR (уникальные):', out_of_range_left.unique())
                if len(out_of_range_right) > 0:
                    print('|    Справа от IQR (уникальные):', out_of_range_right.unique())
         
        print('|---\n|  Все уникальные значения:', stud[column_name].unique() , '\n')
        

    # end function


In [ ]:

def column_content(column_name_list, more_than = 10):
    '''
        Функция для определения типовых параметров данных, а также определние "замусоренности".
        На вход принимает список столбцов датафрейма
    '''
    # проходим циклом по списку колонок    
    for column_name in column_name_list:
        print('-'*60)
        print('|\t' , 'Отчет по колонке [', column_name, ']', '\t'*5)
        print('-'*60)
        if column_name not in stud.columns:
            print('|  Внимание!!! Указанная колонка не найдена в датасете. \n')
            break

        print('|  Количество уникальных значений:', stud[column_name].nunique())
        print('|  Значений, встретившихся в столбце более', more_than,'раз:', 
              (stud[column_name].value_counts() > more_than).sum())
        print('|  Все уникальные значения:', stud[column_name].unique() , '\n')
        
    # end function


In [ ]:

def column_iqr_range(column_name):
    '''
        Функция для расчета промежутка IQR
    '''
    if column_name in stud.columns and stud[column_name].dtype in ['int', 'float']:
        perc25 = stud[column_name].describe().loc['25%']
        perc75 = stud[column_name].describe().loc['75%']
        iqr = perc75 - perc25
        range_left = perc25 - 1.5 * iqr
        range_right = perc75 + 1.5 * iqr
        return range_left, range_right
    else:
        return None


In [ ]:
def get_boxplot(column):
    '''
        Функция построения бокс-плота
    '''
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='score', 
                data=stud.loc[stud.loc[:, column].isin(stud.loc[:, column].value_counts().index)], ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [ ]:
def get_stat_dif(column):
    ''' 
        Функция проверки, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента
    '''
    cols = stud.loc[:, column].value_counts().index
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(stud.loc[stud.loc[:, column] == comb[0], 'score'], 
                        stud.loc[stud.loc[:, column] == comb[1], 'score']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('[', column,  ']', '- найдены статистически значимые различия')
            break

### Загрузка первичных данных

In [ ]:
# файл исходных данных
incoming_files = 'stud_math.csv'

# проверка файла на наличие 
if incoming_files not in os.listdir():
    print('\n!!! ВНИМАНИЕ !!!'*5)
    print('Не найден входной файл данных:', incoming_files)
    # для работы через google colab
    incoming_files = r'/content/drive/My Drive/Colab Notebooks/module_2/stud_math.csv'
else:
    print(r'[OK] Успешно пройдена проверка наличия входного файла датасета.')


In [ ]:
# читаем данные
stud = pd.read_csv(incoming_files)

# получаем характеристики
row_count, col_count = stud.shape

In [ ]:
print('Количество строк датасета:', row_count,'\nКоличество столбцов датасета:', col_count)

In [ ]:
# показывать больше строк
pd.set_option('display.max_rows', 50)

# показывать больше колонок
pd.set_option('display.max_columns', col_count + 5) 

### Первичный осмотр данных

In [ ]:
stud.sample(9)

In [ ]:
stud.info()

In [ ]:
stud.columns

В названиях колонок встречается написание с большой буквы, далее приведем все имена колонок к нижнему регистру.

### Первичная обработка данных

In [ ]:
# Удаляем столбец не заявленный в описании
if 'studytime, granular' in stud.columns:
    stud.drop(['studytime, granular'], inplace=True, axis = 1)

# приводим все названия колонок к нижнему регистру
stud.columns = [str.lower(x) for x in stud.columns]

# получаем список всех колонок
all_columns = stud.columns

In [ ]:
# dtypes: float64(11), int64(1), object(17)

dig_columns = ['age','medu', 'fedu', 'traveltime', 'studytime', 'failures',
              'famrel', 'freetime', 'goout', 'health', 'absences', 'score']

stud.loc[:, dig_columns].info()

In [ ]:
# формируем список строковых столбцов
other_columns = [x for x in all_columns if x not in dig_columns]
other_columns

In [ ]:
# произведем замену пустых значений и значений nan в строковых столбцах
for col_name in other_columns:
    stud[col_name] = stud[col_name].apply(lambda x : None if pd.isnull(x)
                                    else None if str.lower(x) == 'nan'
                                    else None if x.strip == ''
                                    else x.strip())

In [ ]:
# выведем результат первичной обработки
stud.sample(9)

## Распределение признака для числовых переменных

In [ ]:
# количество числовых столбцов
len(dig_columns)

In [ ]:
# список числовых столбцов
dig_columns

In [ ]:
# список и расширенная информация для числовых столбцов
stud.loc[:, dig_columns].info()

### age

age — возраст ученика (от 15 до 22)

In [ ]:
column_content_analysis(['age'])

* Колонка не содержит пустых значений.
* Есть 1 выброс данных.
* Значение выброса `22` в ходит допустимый диапазон, поэтому фильтрация не потребуется.

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['age'].value_counts()).sort_index())

In [ ]:
stud['age'].describe()

In [ ]:
stud['age'].hist()

### medu

Medu — образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

In [ ]:
column_content_analysis(['medu'])

* Колонка содержит только 3 пустых значения. 
* Качество данных хорошее.
* Выбросов не обнаружено.

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['medu'].value_counts()))

In [ ]:
stud['medu'].describe()

In [ ]:
stud['medu'].hist()

### fedu

Fedu — образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

In [ ]:
column_content_analysis(['fedu'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['fedu'].value_counts()))

In [ ]:
stud['fedu'].describe()

* Колонка содержит пустые значения - `24` шт. 
* Качество данных нормальное.
* Обнаружен выброс данных (значение `40`). Это явная опечатка, можно скорректировать до `4`.
* Значение `0`, меньшей левой границы - попадае в рамки начальных условий.

In [ ]:
stud['fedu'].hist()

### traveltime

traveltime — время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)

In [ ]:
column_content_analysis(['traveltime'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['traveltime'].value_counts()))

In [ ]:
stud['traveltime'].describe()

* Колонка содержит пустые значения - 28 шт. 
* Качество данных нормальное.
* Обнаружен выброс данных, но в пределах допустимого.

In [ ]:
stud['traveltime'].hist()

### studytime

studytime — время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)

In [ ]:
column_content_analysis(['studytime'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['studytime'].value_counts()))

In [ ]:
stud['studytime'].describe()

* Колонка содержит пустые значения - 7 шт. 
* Качество данных хорошее.
* Обнаружен выброс данных, но в пределах допустимого.

In [ ]:
stud['studytime'].hist()

### failures

 failures — количество внеучебных неудач (n, если 1<=n<=3, иначе 0)

In [ ]:
column_content_analysis(['failures'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['failures'].value_counts()))

In [ ]:
stud['failures'].describe()

* Колонка содержит пустые значения - 22 шт. 
* Качество данных нормальное.
* Обнаружен выброс данных, но в пределах допустимого.
* Уникальные значения в пределах начальных условий.

In [ ]:
stud['failures'].hist()

### famrel

famrel — семейные отношения (от 1 - очень плохо до 5 - очень хорошо)

In [ ]:
column_content_analysis(['famrel'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['famrel'].value_counts()))

In [ ]:
stud['famrel'].describe()

* Колонка содержит пустые значения - 27 шт. 
* Качество данных нормальное.
* Обнаружен выброс данных, за пределами допустимого со значением `-1`. Возможно опечатка (лишний минус).
* Уникальные значения в пределах начальных условий.

In [ ]:
stud['famrel'].hist()

### freetime

freetime — свободное время после школы (от 1 - очень мало до 5 - очень мого)

In [ ]:
column_content_analysis(['freetime'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['freetime'].value_counts()))

In [ ]:
stud['freetime'].describe()

* Колонка содержит пустые значения - 11 шт. 
* Качество данных хорошее.
* Обнаружен выброс данных, но в пределах допустимого.
* Уникальные значения в пределах начальных условий.

In [ ]:
stud['freetime'].hist()

### goout

goout — проведение времени с друзьями (от 1 - очень мало до 5 - очень много)

In [ ]:
column_content_analysis(['goout'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['goout'].value_counts()))

In [ ]:
stud['goout'].describe()

* Колонка содержит пустые значения - 8 шт. 
* Качество данных хорошее.
* Выбросов данных не обнаружено.
* Уникальные значения в пределах начальных условий.

In [ ]:
stud['goout'].hist()

### health

health — текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)

In [ ]:
column_content_analysis(['health'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['health'].value_counts()))

In [ ]:
stud['health'].describe()

* Колонка содержит пустые значения - 15 шт. 
* Качество данных хорошее.
* Данные не содержат выбросов.
* Уникальные значения в пределах начальных условий.

In [ ]:
stud['health'].hist()

### absences

absences — количество пропущенных занятий

In [ ]:
column_content_analysis(['absences'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
display(pd.DataFrame(stud['absences'].value_counts()).sort_index(ascending=False))

In [ ]:
stud['absences'].describe()

* Колонка содержит пустые значения - 12 шт. 
* Качество данных приемлемое.
* Обнаружен сильный выброс данных, значения `385` и `212` - явно аномальные, поэтому будут исключены.

In [ ]:
stud['absences'].hist()

### score

score — баллы по госэкзамену по математике

In [ ]:
column_content_analysis(['score'])

In [ ]:
# посмотрим на содержимое столбца и количество значений
score_df = pd.DataFrame(stud['score'].value_counts())
score_df.sort_index()

Итак, резульататы госэкзамена:
* 37 учеников получили 0 баллов, 1 ученик получил максимальный балл: 100
* среднее значение 52.3

*Ремарка:*
* менее 27 баллов получили 45 учеников, если применить минимальный балл по ЕГЭ по РФ, то эти ученики предположительно не набрали пороговый балл для аттестата.

In [ ]:
stud['score'].describe()

* Колонка содержит пустые значения - 6 шт. 
* Качество данных хорошее. Распределние баллов показывает, что используется 100-бальная шкала оценок.
* Выбросы данных не обнаружены.
* Уникальные значения в пределах начальных условий.


In [ ]:
stud['score'].hist()

Выводы по результатам осмотра данных - сделующие столбцы требуют дополнительной обработки:

*   `fedu` - корректировка значения
*   `absences` - устранение выбросов

Почти все столбцы (кроме `age`) имеют пропуски данных, поэтому строки с пропусками будут исключены из анализа.

Топ 3 столбцов - лидеров по пропуску данных:
* `traveltime`
* `famrel`
* `fedu`

## Распределение признака для строковых переменных

In [ ]:
# количество текстовых столбцов
len(other_columns)

In [ ]:
# список текстовых столбцов
other_columns

In [ ]:
# список и расширенная информация для стоковых столбцов
stud.loc[:, other_columns].info()

### school

school — аббревиатура школы, в которой учится ученик

In [ ]:
column_content_analysis(['school'])

In [ ]:
stud['school'].describe()

In [ ]:
# посмотрим на содержимое столбца
display(pd.DataFrame(stud['school'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - отсутсвуют
* засоренность данных - отсутсвует
* специальные символы в данных не найдены


### sex

sex — пол ученика ('F' - женский, 'M' - мужской)

In [ ]:
column_content_analysis(['sex'])
stud['sex'].describe()

In [ ]:
display(pd.DataFrame(stud['sex'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - нет
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### address

address — тип адреса ученика ('U' - городской, 'R' - за городом)

In [ ]:
column_content_analysis(['address'])
stud['address'].describe()

In [ ]:
display(pd.DataFrame(stud['address'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 17 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### famsize

famsize — размер семьи('LE3' <= 3, 'GT3' >3)

In [ ]:
# famsize
column_content_analysis(['famsize'])
stud['famsize'].describe()

In [ ]:
display(pd.DataFrame(stud['famsize'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 27 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### pstatus

pstatus — статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно)

In [ ]:
# pstatus
column_content_analysis(['pstatus'])
stud['pstatus'].describe()

In [ ]:
display(pd.DataFrame(stud['pstatus'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 45 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### mjob

Mjob — работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

In [ ]:
# mjob
column_content_analysis(['mjob'])
stud['mjob'].describe()

In [ ]:
display(pd.DataFrame(stud['mjob'].value_counts()))

* количесто уникальных данных - 5 шт
* пустые значения - 19 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### fjob

Fjob — работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

In [ ]:
# fjob
column_content_analysis(['fjob'])
stud['fjob'].describe()

In [ ]:
display(pd.DataFrame(stud['fjob'].value_counts()))

* количесто уникальных данных - 5 шт
* пустые значения - 36 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### reason

reason — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое)

In [ ]:
# reason
column_content_analysis(['reason'])
stud['reason'].describe()

In [ ]:
display(pd.DataFrame(stud['reason'].value_counts()))

* количесто уникальных данных - 4 шт
* пустые значения - 17 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### guardian

guardian — опекун ('mother' - мать, 'father' - отец, 'other' - другое)

In [ ]:
# guardian
column_content_analysis(['guardian'])
stud['guardian'].describe()

In [ ]:
display(pd.DataFrame(stud['guardian'].value_counts()))

* количесто уникальных данных - 3 шт
* пустые значения - 31 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### schoolsup

schoolsup — дополнительная образовательная поддержка (yes или no)

In [ ]:
# schoolsup
column_content_analysis(['schoolsup'])
stud['schoolsup'].describe()

In [ ]:
display(pd.DataFrame(stud['schoolsup'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 9 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### famsup

famsup — семейная образовательная поддержка (yes или no)

In [ ]:
# famsup
column_content_analysis(['famsup'])
stud['famsup'].describe()

In [ ]:
display(pd.DataFrame(stud['famsup'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 39 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### paid

paid — дополнительные платные занятия по математике (yes или no)

In [ ]:
# paid
column_content_analysis(['paid'])
stud['paid'].describe()

In [ ]:
display(pd.DataFrame(stud['paid'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 40 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### activities

activities — дополнительные внеучебные занятия (yes или no)

In [ ]:
# activities
column_content_analysis(['activities'])
stud['activities'].describe()

In [ ]:
display(pd.DataFrame(stud['activities'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 14 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### nursery

nursery — посещал детский сад (yes или no)

In [ ]:
# nursery
column_content_analysis(['nursery'])
stud['nursery'].describe()

In [ ]:
display(pd.DataFrame(stud['nursery'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 16 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### higher

higher — хочет получить высшее образование (yes или no)

In [ ]:
# higher
column_content_analysis(['higher'])
stud['higher'].describe()

In [ ]:
display(pd.DataFrame(stud['higher'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 20 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### internet

internet — наличие интернета дома (yes или no)

In [ ]:
# internet
column_content_analysis(['internet'])
stud['internet'].describe()

In [ ]:
display(pd.DataFrame(stud['internet'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 34 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

### romantic

romantic — в романтических отношениях (yes или no)

In [ ]:
# romantic
column_content_analysis(['romantic'])
stud['romantic'].describe()

In [ ]:
display(pd.DataFrame(stud['romantic'].value_counts()))

* количесто уникальных данных - 2 шт
* пустые значения - 31 шт
* засоренность данных - отсутсвует
* специальные символы в данных не найдены

## Устранение выбросов

### fedu

Посмотрим на строку с выбросом внимательней


In [ ]:
stud[stud['fedu'] == 40]

Значение `40` с большой вероятностей опечатка,  предположим что должно быть значение `4`

In [ ]:
# исправим ошибочное значение
stud['fedu'][stud['fedu'] == 40] = 4

#  проверим результат 
stud['fedu'].loc[11]

### absences

* Обнаружен сильный выброс данных, значения `385` и `212` - явно аномальные, поэтому будут исключены.

In [ ]:
stud[stud['absences'] >= 212]

In [ ]:
stud[stud['absences'] >= 212].index


In [ ]:
stud.drop(stud[stud['absences'] >= 212].index, inplace=True)

## Оценка количества уникальных значений для номинативных переменных

In [ ]:
# количество номинативных переменных
len(other_columns)

In [ ]:
# список номинативных переменных
other_columns

In [ ]:
column_content(other_columns)

## Преобразование данных

Заполнить пропуски на данный момент не представляется возможным, так как методика не предлагается. Поэтому отбросим данные с пропусками у числовых столбцов.

In [ ]:
# размерность исходного датасета
stud.shape

In [ ]:
# удаление лишних строк
stud.dropna('index', subset=dig_columns, inplace=True)

In [ ]:
# размерность датасета после обработки
stud.shape

Датасет сократился с 393 строк до 261.

## Корреляционный анализ количественных элементов

Корреляции рассмотрим только для числовых столбцов. Выясним какие параметры коррелируют с итоговой оценкой `score`

In [ ]:
sns.pairplot(stud[dig_columns], kind = 'reg')

Выведем матрицу корреляции

In [ ]:
stud[dig_columns].corr()

In [ ]:
# расмотрим корреляцию score с другими параметрами
stud[dig_columns].corr()['score'].sort_values()

Выводы на основе корреляционной связи параметров на результат успеваимости по математике:
* положительно влияет на итоговый бал - образование матери (`medu`) и отца (`fedu`), а также время выделенное на учебу (`studytime`) помимо школы. 
* на снижение результата влияет количество пропущенных занятий `absences`, свободное время после школы `freetime`, а также семейные отношения `famrel`.
* снижение успеваемости также связано с увеличением количества внеучебных неудач `failures`и количество проведенного времени с друзьями `goout`. Наблюдаетя также снижение результата успеваемости при увеличении возраста учащегося (`age`)

* отрицательная корреляция с score у параметров: `failures`, `goout`, `age`, `traveltime`, `health`
* положительная корреляция с score у параметров: `famrel`, `freetime`, `absences`, `studytime`, `fedu`, `medu`

## Отбор не коррелирующих переменных

*Справочно*

* Если коэффициент корреляции отрицательный, это означает наличие противоположной связи: чем выше значение одной переменной, тем ниже значение другой.
* Сила связи характеризуется также и абсолютной величиной коэффициента корреляции. 
* Для словесного описания величины коэффициента корреляции используются следуюшие градации:

Интерпретация значений:
* до 0,2 - очень слабая корреляция
* до 0,5 - слабая корреляция
* до 0,7 - средняя корреляция
* до 0,9 - высокая корреляция
* свыше 0,9	- очень высокая корреляция

**Выбор значимого коэффициента корреляции**

Так как выборка достаточно большая, то можно считать что переменные не связаны если их корреляция попадает в диапазон `[-0.5 , 0.5]`

In [ ]:
# сформируем датафрейм
kk_df = pd.DataFrame(stud[dig_columns].corr()['score'].sort_values())

# отфильтруем
kk_df.query('-0.5 <= score <= 0.5')

Таким образом для модели будем использовать указанные столбцы:

In [ ]:
kk_df.query('-0.5 <= score <= 0.5').index

In [ ]:
# сохраним в список колонки для модели
column_for_model = ['failures', 'goout', 'age', 'traveltime', 'health', 'famrel',
       'freetime', 'absences', 'studytime', 'fedu', 'medu']

In [ ]:

# column_for_model0 = ['traveltime', 'health', 'famrel', 'freetime', 'absences']

## Анализ номинативных переменных, устранение не влияющих на предсказание конечной величины

Для номинативных переменных использовать корреляционный анализ не получится. Однако можно посмотреть, различаются ли распределения результата успеваемости в зависимости от значения этих переменных.

In [ ]:
# Список номинативных переменных для рассмотрения
other_columns

Построим графики плотности распределения переменных

In [ ]:
for col in other_columns:
    get_boxplot(col)

По графикам похоже, что все параметры, кроме `sex`, `address`, `famsize` не могут влиять на оценку.

Удостоверимся в этом. Настоящую значимость различий может помочь распознать статистика. Проверим, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента. Проанализируем нулевую гипотезу о том, что распределения успеваемости по различным параметрам неразличимы:

In [ ]:
for col in other_columns:
    get_stat_dif(col)

Как мы видим, серьёзно отличаются три параметра: 
* `sex` - пол ученика
* `address` - тип адреса ученика
* `famsize` - размер семьи

Оставим эти переменные в датасете для дальнейшего построения модели.

In [ ]:
column_for_model

In [ ]:
column_for_model.append('sex')
column_for_model.append('address')
column_for_model.append('famsize')

column_for_model

## Выводы о качестве данных

В результате EDA анализа влияния условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска, были получены следующие выводы:






* в данных не слишком много пропущенных значений, % полноты данных не снижался ниже 88, содержимое данных не содержит мусорных символов. Детальные данные доступны в блоке анализа каждого отдельного столбца.

Выводы по результатам осмотра данных - некоторые столбцы требуют дополнительной обработки:

*   `fedu` - корректировка значения
*   `absences` - устранение выбросов
* `famrel` - обнаружен выброс данных, за пределами допустимого со значением `-1`. Строка будет исключена из анализа.

Многие столбцы (кроме `age`, `school`, `sex`) имеют пропуски данных, но только  строки с пропусками для числовых столбцов были исключены из анализа.

Топ 3 столбцов - лидеров по пропуску данных:
* `traveltime` - 28 значений
* `famrel` - 27 значений
* `fedu` - 24 значения

## Выводы о данных, которые подходят для построения дальнейшей модели

 Итак, в нашем случае важные переменные, которые, возможно, оказывают влияние на оценку:

In [ ]:
# количество колонок
len(column_for_model)

In [ ]:
# наименование колонок
column_for_model

In [ ]:
# пример данных
stud_model = stud[column_for_model]
stud_model.sample(10)

In [ ]:
# размерность итоговой модели - строк, колонок
stud_model.shape

Выводы на основе корреляционной связи параметров на результат успеваимости по математике:
* положительно влияет на итоговый бал - образование матери (`medu`) и отца (`fedu`), а также время выделенное на учебу помимо школы (`studytime` ). 
* на снижение результата влияет количество пропущенных занятий `absences`, свободное время после школы `freetime`, а также семейные отношения `famrel`.
* снижение успеваемости также связано с увеличением количества внеучебных неудач `failures`и количество проведенного времени с друзьями `goout`. Наблюдаетя также снижение результата успеваемости при увеличении возраста учащегося (`age`)

Важные параметры, которые предлагаю использовать в дальнейшем для построения модели:
* `failures`
* `goout`
* `age`
* `traveltime`
* `health`
* `famrel`
* `freetime`
* `absences`
* `studytime`
* `fedu`
* medu
* `sex`
* `address`
* `famsize`

Итоговая модель позволит отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.


# Спасибо за внимание!